In [19]:
from PIL import Image
import torch
import open_clip
import pandas as pd

# Load CLIP model
model, preprocess, tokenizer = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')
tokenizer = open_clip.get_tokenizer('ViT-B-32')

In [ ]:
class_descriptions = [
    # 🔵 Nervous System
    "An iris with a distorted pupil indicating neurological imbalance",
    "An iris with concentric stress rings indicating chronic anxiety or emotional overload",
    "An iris with signs of mental fatigue and tight fiber structure",
    "An iris showing radial furrows and stress lines from chronic emotional pressure",

    # 🟢 Digestive System
    "An iris with signs of sluggish colon and digestive insufficiency",
    "An iris with a pale pancreas zone and faded stomach ring",
    "An iris indicating bloating or gas from transverse colon stress",
    "An iris showing signs of poor nutrient absorption in the small intestine",

    # 🔴 Circulatory System
    "An iris with a visible cholesterol ring and signs of arteriosclerosis",
    "An iris showing blue or grey rings indicating low oxygenation",
    "An iris with signs of poor circulation and lymph congestion",
    "An iris showing weak hemoglobin absorption or low iron levels",

    # 🟡 Endocrine System
    "An iris showing thyroid imbalance affecting metabolism or mood",
    "An iris with signs of adrenal exhaustion and fatigue",
    "An iris with pigmentation in the ovarian zone indicating hormonal imbalance",
    "An iris with signs of insulin resistance and pancreas weakness",
    "An iris showing pituitary stress affecting hormonal regulation",

    # 🟣 Musculoskeletal System
    "An iris showing signs of weakness in spine, legs, or joint areas",
    "An iris showing pigmentation near eye area indicating neck, spine, or posture tension",

    # 🟠 Lymphatic / Immune System
    "An iris showing lymphatic congestion in the lymphatic rosary",
    "An iris with signs of immune overload from toxin buildup",

    # 🌬️ Respiratory System
    "An iris showing pigmentation in the lung zone suggesting toxin buildup or shallow breathing",

    # 💧 Urinary System
    "An iris with irritation markers near the bladder or urethral zone",

    # ⚪ Vitality & General
    "An iris with dark radial pigments suggesting toxin accumulation near pupil",
    "An iris showing overall low vitality and burnout",
    "A healthy iris with clear fibers and no stress rings",
]


In [38]:
# Load and preprocess the image
image_path = "images/5.jpg"  # <-- Change this to your image path
image = Image.open(image_path).convert("RGB")
image_input = preprocess(image).unsqueeze(0)

# Prepare model and inputs
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
image_input = image_input.to(device)
text_inputs = tokenizer(class_descriptions).to(device)

# Run inference
with torch.no_grad():
    image_features = model.encode_image(image_input)
    text_features = model.encode_text(text_inputs)

    # Normalize embeddings
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    # Compute similarity and probabilities
    similarity = (100.0 * image_features @ text_features.T)
    probs = similarity.softmax(dim=-1)[0]

# Create results as list of dicts
results = [
    {
        "Health Pattern": desc,
        "Confidence Score (%)": float(f"{probs[i].item()*100:.2f}"),
        "Short Note": desc.split("with")[1].strip().capitalize() if "with" in desc else desc
    }
    for i, desc in enumerate(class_descriptions)
]

# Create and display sorted DataFrame
df = pd.DataFrame(results)
df_sorted = df.sort_values(by="Confidence Score (%)", ascending=False).reset_index(drop=True)

# Display top 5 results
print("\n--- Iris Health Analysis (Top 5) ---")
print(image_path)
topk = 10
top_indices = probs.topk(topk).indices

for idx in top_indices:
    pattern = class_descriptions[idx]
    score = probs[idx].item() * 100
    short_note = pattern.split("with")[1].strip().capitalize() if "with" in pattern else pattern
    print(f"- {short_note}: {score:.2f}% confidence")


--- Iris Health Analysis (Top 5) ---
images/5.jpg
- Dark spots or pigmentation near pupil and sclera: 25.54% confidence
- A distorted pupil indicating neurological imbalance: 22.60% confidence
- A visible cholesterol ring and signs of arteriosclerosis: 14.90% confidence
- An iris showing pigmentation near eye area indicating neck, spine, or posture tension: 5.30% confidence
- Signs of mental fatigue and tight fiber structure: 5.08% confidence
- Signs of insulin resistance and pancreas weakness: 4.57% confidence
- Clear fibers and no stress rings: 3.98% confidence
- An iris indicating bloating or gas from transverse colon stress: 3.89% confidence
- An iris showing overall low vitality and burnout: 3.13% confidence
- Visible stress rings showing chronic anxiety or emotional stress: 2.62% confidence


## old prompt


In [27]:
# Load and preprocess your iris image
image = Image.open("images/5.jpg").convert("RGB")  # Change this to your image filename
image_input = preprocess(image).unsqueeze(0)  # [1, 3, 224, 224]

# Improved prompt list based on your dataset
class_descriptions = [
    # Digestive
    "An iris with a pale pancreas zone and faded stomach ring",
    "An iris with stress signs in the small intestine and sluggish colon",
    "An iris showing liver and gallbladder congestion",
    "An iris with signs of poor absorption and digestive burden",

    # Circulatory
    "An iris with a visible cholesterol ring and arteriosclerosis",
    "An iris with vein and lymph congestion and blue hues",
    "An iris showing poor blood flow and circulation issues",

    # Nervous
    "An iris with deep stress rings and emotional stress",
    "An iris with signs of mental fatigue and tight fiber structure",
    "An iris with a distorted pupil indicating neurological imbalance",

    # Endocrine
    "An iris showing adrenal fatigue and thyroid imbalance",
    "An iris with pancreas weakness and possible insulin resistance",
    "An iris with signs of hormonal imbalance and low vitality",

    # Musculoskeletal
    "An iris with markers for knee and spinal tension",
    "An iris showing signs of neck strain and poor posture alignment",

    # Respiratory / Lymphatic
    "An iris with toxic accumulation in the lung zone",
    "An iris with lymphatic rosary and mucous buildup"
]

# Tokenize the text
text_inputs = tokenizer(class_descriptions)

# Move to device
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
image_input = image_input.to(device)
text_inputs = text_inputs.to(device)

# Run CLIP
with torch.no_grad():
    image_features = model.encode_image(image_input)
    text_features = model.encode_text(text_inputs)

    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    similarity = (100.0 * image_features @ text_features.T)
    probs = similarity.softmax(dim=-1)

# Display results clearly
print("\n--- Iris Health Analysis ---")
topk = 5  # Show top 5 most likely results
top_probs, top_indices = probs[0].topk(topk)

for i in range(topk):
    index = top_indices[i].item()
    score = top_probs[i].item()
    print(f"- {class_descriptions[index]}: {score:.2%} confidence")



--- Iris Health Analysis ---
- An iris with a distorted pupil indicating neurological imbalance: 53.59% confidence
- An iris with a visible cholesterol ring and arteriosclerosis: 14.88% confidence
- An iris with signs of mental fatigue and tight fiber structure: 9.64% confidence
- An iris with signs of hormonal imbalance and low vitality: 8.15% confidence
- An iris with pancreas weakness and possible insulin resistance: 8.02% confidence


In [ ]:
# Define refined health prompts
class_descriptions = [
    # 🔵 Nervous System
    "An iris with a distorted pupil indicating neurological imbalance",
    "An iris with visible stress rings showing chronic anxiety or emotional stress",
    "An iris with signs of mental fatigue and tight fiber structure",

    # 🟢 Digestive System
    "An iris with a pale pancreas zone and faded stomach ring",
    "An iris showing underactive stomach with weak digestive zone",

    # 🔴 Circulatory System
    "An iris with a visible cholesterol ring and arteriosclerosis",
    "An iris with signs of poor circulation and lymph congestion",
    "An iris showing stress in the heart zone from emotional strain",
    "An iris showing signs of cardiovascular stress or heart tension",

    # 🟡 Endocrine System
    "An iris showing thyroid stress and hormonal imbalance",
    "An iris with signs of adrenal exhaustion and fatigue",
    "An iris indicating signs of hormonal imbalance and low vitality",

    # 🟣 Musculoskeletal System
    "An iris showing signs of weakness in neck, spine, or knee regions",

    # 🟠 Reproductive System
    "An iris showing signs of ovarian zone congestion or menstrual imbalance",

    # ⚪ Vitality & General
    "A healthy iris with clear fibers and no stress rings",
    "An iris with signs of low vitality or burnout",
    "An iris with dark spots or discoloration around the pupil and sclera",
    "An iris with blue or grey rings indicating low oxygenation",
]



# Load and preprocess the image
image_path = "images/12.jpg"  # <-- Change this to your image path
image = Image.open(image_path).convert("RGB")
image_input = preprocess(image).unsqueeze(0)

# Prepare model and inputs
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
image_input = image_input.to(device)
text_inputs = tokenizer(class_descriptions).to(device)

# Run inference
with torch.no_grad():
    image_features = model.encode_image(image_input)
    text_features = model.encode_text(text_inputs)

    # Normalize embeddings
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    # Compute similarity and probabilities
    similarity = (100.0 * image_features @ text_features.T)
    probs = similarity.softmax(dim=-1)[0]

# Create results as list of dicts
results = [
    {
        "Health Pattern": desc,
        "Confidence Score (%)": float(f"{probs[i].item()*100:.2f}"),
        "Short Note": desc.split("with")[1].strip().capitalize() if "with" in desc else desc
    }
    for i, desc in enumerate(class_descriptions)
]

# Create and display sorted DataFrame
df = pd.DataFrame(results)
df_sorted = df.sort_values(by="Confidence Score (%)", ascending=False).reset_index(drop=True)

# Display top 5 results
print("\n--- Iris Health Analysis (Top 5) ---")
topk = 5
top_indices = probs.topk(topk).indices

for idx in top_indices:
    pattern = class_descriptions[idx]
    score = probs[idx].item() * 100
    short_note = pattern.split("with")[1].strip().capitalize() if "with" in pattern else pattern
    print(f"- {short_note}: {score:.2f}% confidence")